<h1> 2. Creating a sampled dataset </h1>

This notebook illustrates:
<ol>
<li> Sampling a BigQuery dataset to create datasets for ML
<li> Preprocessing with Pandas
</ol>

In [1]:
# change these to try this notebook out
BUCKET = 'cloud-training-demos-ml'
PROJECT = 'cloud-training-demos'
REGION = 'us-central1'

In [2]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [3]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

<h2> Create ML dataset by sampling using BigQuery </h2>
<p>
Let's sample the BigQuery data to create smaller datasets.
</p>

In [1]:
# Create SQL query using natality data after the year 2000
import google.datalab.bigquery as bq
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
"""

There are only a limited number of years and months in the dataset. Let's see what the hashmonths are.

In [2]:
# Call BigQuery but GROUP BY the hashmonth and see number of records for each group to enable us to get the correct train and evaluation percentages
df = bq.Query("SELECT hashmonth, COUNT(weight_pounds) AS num_babies FROM (" + query + ") GROUP BY hashmonth").execute().result().to_dataframe()
print("There are {} unique hashmonths.".format(len(df)))
df.head()

There are 96 unique hashmonths.


,hashmonth,num_babies
0,3408502330831153141,323970
1,1088037545023002395,362125
2,1525201076796226340,303664
3,4979697502521811334,357965
4,411066950820961322,318834


Here's a way to get a well distributed portion of the data in such a way that the test and train sets do not overlap:

In [3]:
# Added the RAND() so that we can now subsample from each of the hashmonths to get approximately the record counts we want
trainQuery = "SELECT * FROM (" + query + ") WHERE MOD(hashmonth, 4) < 3 AND RAND() < 0.0005"
evalQuery = "SELECT * FROM (" + query + ") WHERE MOD(hashmonth, 4) = 3 AND RAND() < 0.0005"
traindf = bq.Query(trainQuery).execute().result().to_dataframe()
evaldf = bq.Query(evalQuery).execute().result().to_dataframe()
print("There are {} examples in the train dataset and {} in the eval dataset".format(len(traindf), len(evaldf)))

There are 13307 examples in the train dataset and 3269 in the eval dataset


<h2> Preprocess data using Pandas </h2>
<p>
Let's add extra rows to simulate the lack of ultrasound. In the process, we'll also change the plurality column to be a string.

In [4]:
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,7.517763,True,12,1,38.0,7170969733900686954
1,8.999270,True,13,1,40.0,7108882242435606404
2,6.596231,False,14,1,39.0,5934265245228309013
3,6.250105,True,14,1,39.0,1525201076796226340
4,6.000983,False,14,1,39.0,1443901198490054949


Also notice that there are some very important numeric fields that are missing in some rows (the count in Pandas doesn't count missing data)

In [5]:
# Let's look at a small sample of the training data
traindf.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hashmonth
count,13294.000000,13307.000000,13307.000000,13211.000000,1.330700e+04
mean,7.220347,27.309386,1.036973,38.591931,4.382953e+18
std,1.329478,6.224671,0.198031,2.640096,2.801429e+18
min,0.500449,12.000000,1.000000,17.000000,1.244589e+17
25%,6.563162,22.000000,1.000000,38.000000,1.525201e+18
50%,7.312733,27.000000,1.000000,39.000000,4.329667e+18
75%,8.062305,32.000000,1.000000,40.000000,7.108882e+18
max,11.812368,50.000000,3.000000,47.000000,9.183606e+18


In [6]:
# It is always crucial to clean raw data before using in ML, so we have a preprocessing step
import pandas as pd
def preprocess(df):
  # clean up data we don't want to train on
  # in other words, users will have to tell us the mother's age
  # otherwise, our ML service won't work.
  # these were chosen because they are such good predictors
  # and because these are easy enough to collect
  df = df[df.weight_pounds > 0]
  df = df[df.mother_age > 0]
  df = df[df.gestation_weeks > 0]
  df = df[df.plurality > 0]
  
  # modify plurality field to be a string
  twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
  df['plurality'].replace(twins_etc, inplace=True)
  
  # now create extra rows to simulate lack of ultrasound
  nous = df.copy(deep=True)
  nous.loc[nous['plurality'] != 'Single(1)', 'plurality'] = 'Multiple(2+)'
  nous['is_male'] = 'Unknown'
  
  return pd.concat([df, nous])

In [7]:
traindf.head()# Let's see a small sample of the training data now after our preprocessing
traindf = preprocess(traindf)
evaldf = preprocess(evaldf)
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,7.517763,True,12,Single(1),38.0,7170969733900686954
1,8.999270,True,13,Single(1),40.0,7108882242435606404
2,6.596231,False,14,Single(1),39.0,5934265245228309013
3,6.250105,True,14,Single(1),39.0,1525201076796226340
4,6.000983,False,14,Single(1),39.0,1443901198490054949


In [12]:
traindf.tail()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
15077,8.562754,Unknown,47,Single(1),39.0,-4701948054522374613
15078,9.735613,Unknown,47,Single(1),39.0,124458947937444850
15080,9.539402,Unknown,49,Single(1),39.0,8599690069971956834
15081,3.249614,Unknown,49,Multiple(2+),31.0,1148502204931914436
15082,4.750962,Unknown,49,Single(1),36.0,7895641210289919997


In [8]:
# Describe only does numeric columns, so you won't see plurality
traindf.describe()

,weight_pounds,mother_age,gestation_weeks,hashmonth
count,26400.000000,26400.000000,26400.000000,2.640000e+04
mean,7.221361,27.313258,38.597424,4.382000e+18
std,1.328881,6.225467,2.620030,2.801119e+18
min,0.500449,12.000000,17.000000,1.244589e+17
25%,6.563162,22.000000,38.000000,1.525201e+18
50%,7.312733,27.000000,39.000000,4.329667e+18
75%,8.062305,32.000000,40.000000,7.108882e+18
max,11.812368,50.000000,47.000000,9.183606e+18


<h2> Write out </h2>
<p>
In the final versions, we want to read from files, not Pandas dataframes. So, write the Pandas dataframes out as CSV files. 
Using CSV files gives us the advantage of shuffling during read. This is important for distributed training because some workers might be slower than others, and shuffling the data helps prevent the same data from being assigned to the slow workers.


In [9]:
traindf.to_csv('train.csv', index=False, header=False)
evaldf.to_csv('eval.csv', index=False, header=False)

In [10]:
%bash
wc -l *.csv
head *.csv
tail *.csv

   6508 eval.csv
  26400 train.csv
  32908 total
==> eval.csv <==
4.9163084426,False,16,Single(1),36.0,4740473290291881219
6.4992274837599995,True,16,Single(1),38.0,4740473290291881219
7.12534030784,True,16,Single(1),41.0,4740473290291881219
6.79244229222,False,17,Single(1),34.0,4740473290291881219
6.0009827716399995,True,17,Single(1),38.0,4740473290291881219
7.7492485093,True,18,Single(1),39.0,4740473290291881219
6.8122838958,True,18,Single(1),38.0,4740473290291881219
3.69053826588,True,18,Single(1),32.0,4740473290291881219
6.1244416383599996,False,18,Single(1),41.0,4740473290291881219
2.25091969502,True,19,Single(1),31.0,4740473290291881219

==> train.csv <==
7.5177631342,True,12,Single(1),38.0,7170969733900686954
8.99926953484,True,13,Single(1),40.0,7108882242435606404
6.59623087904,False,14,Single(1),39.0,5934265245228309013
6.2501051276999995,True,14,Single(1),39.0,1525201076796226340
6.0009827716399995,False,14,Single(1),39.0,1443901198490054949
7.6279942652,True,14,Single(1),40.

Copyright 2017-2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License